In [ ]:
import tensorflow as tf
import open3d as o3d
import numpy as np
import util
from models.pointnet2 import *

print(tf.__version__)
print(o3d.__version__)

In [ ]:
input_shape = (8, np.random.randint(100), 3)
print(input_shape)
bs = input_shape[0]
num_pts = input_shape[1]

x = tf.random.normal(input_shape)
print("Input Tensor (raw):",x.shape)
x = PointNet2(x.shape, 128)(x)
print("Output Tensor (Encoder):",x.shape)
x = MLP()(x)
print("Output Tensor (Decoder):",x.shape)

x = tf.reshape(x, [8, num_pts, 3])
print("Final Output:", x.shape)

In [ ]:
input_shape = (1, 189, 3)
x = tf.random.normal(input_shape)
print(x)

In [ ]:
input_shape = (1, 12, 189, 3)
bs = input_shape[0]
num_pts = input_shape[1]

# Point functions (MLP implemented as conv2d)
# By default, Keras version of Conv2D has: strides=[1,1], padding="valid"
x = tf.random.normal(input_shape)
x = tf.keras.layers.Conv2D(64, [1,3], data_format="channels_last", activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Conv2D(64, [1,1], data_format="channels_last",activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Conv2D(64, [1,1], data_format="channels_last", activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Conv2D(128, [1,1], data_format="channels_last", activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Conv2D(1024, [1,1], data_format="channels_last", activation='relu')(x)
print(x.shape)

# Symmetric function: max pooling
x = tf.keras.layers.MaxPool2D(pool_size=[num_pts, 1], strides=(2, 2), padding='valid')(x)
print(x.shape)

# MLP on global point cloud vector
x = tf.reshape(x, [bs, -1])
print(x.shape)
x = tf.keras.layers.Dense(1024,activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Dense(512,activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Dense(256,activation='relu')(x)
print(x.shape)

mean, logvar = tf.split(x, num_or_size_splits=2, axis=1)
print("Mean:",mean.shape, " Log(Var):",logvar.shape)